In [1]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(key_param.MONGO_URI)
dbName = "PKU"
collectionName = "collection_of_pku_text_blobs"
collection = client[dbName][collectionName]

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key, disallowed_special=())

vectorStore = MongoDBAtlasVectorSearch(collection=collection, embedding=embeddings, index_name='knn')

In [12]:
def query_data(query):
    docs = vectorStore.similarity_search(query, k=1)
    as_output = docs[0].page_content

    llm = OpenAI(openai_api_key=key_param.openai_api_key, temperature=0)
    retriever = vectorStore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.run(query)

    return as_output, retriever_output

In [13]:
docs = vectorStore.similarity_search("Can you describe PKU?", k=1)
docs

[Document(page_content='Phenylketonuria (PKU) is an autosomal recessive inborn error of metabolism resulting from a deficiency of phenylalanine hydroxylase (PAH; 612349), an enzyme that catalyzes the hydroxylation of phenylalanine to tyrosine, the rate-limiting step in phenylalanine catabolism. If undiagnosed and untreated, phenylketonuria can result in impaired postnatal cognitive development resulting from a neurotoxic effect of hyperphenylalaninemia (Zurfluh et al., 2008).\n\nSee Scriver (2007) and Blau et al. (2010) for detailed reviews of PKU.', metadata={'_id': ObjectId('65a964fdae5cf51da363e067'), 'embedding': [-0.00431883228731077, 0.0026127472207069504, 0.018536207764732377, -0.03455065829848616, -0.046093541959216555, 0.02378769961216085, -0.0022845290966580015, -0.008351691711477281, -0.03330277908449613, -0.005475720347419134, 0.006648856728078941, 0.0027849806758378074, -0.010905944652236115, -0.002320275588135703, -0.019784086978722405, -0.01913414856872885, 0.02823326954